 Primeiramente são importadas as bibliotecass
nescessárias, sendo elas:
 - pandas: para leitura dos dados da base de dados
 - sklearn model_selection: para seleção dos modelos,   neste caso utiliza-se o k-fold apenas
 
    As demais são os métodos de regressão escolhidos:
 - Regressão Linear
 - Elastic Net
 - Arvore de Decisão (Regressão)
 - Floresta Aleatória (Regressão)
 - Boosting de gradiente (Regressão)
 - Support Vector (Regressão)

In [163]:
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR


Então realiza-se a leitura da base de dados contida no arquivo 'Analise Geral Normalizada.xlsx' e seus dados são divididos entre:
- X : dados de entrada da função.
- Y : valor a ser predito

In [164]:
df = pd.read_excel("Analise Geral Normalizada.xlsx")
array = df.values
X = array[:,1:23]
Y = array[:,23]

  Cria-se o modelo k-fold utilizando 12 folds (a
partir de 12 folds não houve melhoria consideravel nos valores de acurácia nas saídas, e se tornam bastante custosos a partir deste ponto)

In [165]:
seed = 1
kfold = model_selection.KFold(n_splits=12, random_state=seed, shuffle=True)

Criam-se os modelos de teste

In [166]:
linearReg = LinearRegression()
elastic = ElasticNet()
tree = DecisionTreeRegressor()
forest = RandomForestRegressor()
boosting = GradientBoostingRegressor()
supportVector = SVR()

Executa o cross validation dos K-folds nos modelos de teste

In [167]:
results = [['Linear Regression:', model_selection.cross_val_score(linearReg, X, Y, cv=kfold)],
           ['Elastic Net:', model_selection.cross_val_score(elastic, X, Y, cv=kfold)],
           ['Decision Tree:', model_selection.cross_val_score(tree, X, Y, cv=kfold)],
           ['Random Forest:', model_selection.cross_val_score(forest, X, Y, cv=kfold)],
           ['Gradient Boosting:', model_selection.cross_val_score(boosting, X, Y, cv=kfold)],
           ['Support Vector:', model_selection.cross_val_score(supportVector, X, Y, cv=kfold)],
]

Analise final dos testes de cross validation, sendo apresentados os dados de acurácia média entre os diferentes modelos

In [168]:
for result in results:
    print(result[0], result[1].mean())

Linear Regression: 0.7775510735612521
Elastic Net: -0.0975090779406706
Decision Tree: 0.9163300422999762
Random Forest: 0.9533228367451723
Gradient Boosting: 0.9486519903018733
Support Vector: 0.6662319997294802


Foram encontrados melhores resultados nos casos de Florestas Aleatória, Boosting de Gradiente e Árvore de Decisão